# 미리보기 & 이미지 인식하기

## 추가 파일 설치

In [1]:
# matplotlib 한글 폰트 깨짐 방지
"""
!sudo apt-get install -y fonts-nanum* | tail -n 1
!sudo fc-cache -fv
!rm -rf ~/.cache/matplotlib
"""


'\n!sudo apt-get install -y fonts-nanum* | tail -n 1\n!sudo fc-cache -fv\n!rm -rf ~/.cache/matplotlib\n'

## 라이브러리 불러오기

In [2]:
# 필요 라이브러리 설치
!pip install torchviz
!apt install tree
!pip install jupyter_contrib_nbextensions && jupyter contrib nbextension install

'apt'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


'jupyter'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [3]:
#%%

In [4]:
# 라이브러리 임포트
#%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as img
from IPython.display import display

# 나눔 고딕 폰트의 경로 명시 (폰트 없으니 생략)
#path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
#font_name = fm.FontProperties(fname=path, size=10).get_name()

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

c:\Users\white\AppData\Local\Programs\Python\Python37\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 초기 설정

In [5]:
#초기설정

#기본 폰트 사이즈 변경
plt.rcParams['font.size']=14

#warning 표시 끄기(사용안함)
"""
import warnings
warnings.simplefilter('ignore')
"""

#GPU 디바이스 할당 (8장에서 자세히 설명)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [6]:
#공통함수 읽어오기 (9강에서 자세히 설명)

#공통함수 다운로드(이책에서 사용할 데이터 셋과 여러 함수들이 들어있음(loss function, learn 등))
#초기에만 해주고 나중부턴 안해도 됨, 코드 자체를 처음실행할때만
#!git clone https://github.com/wikibook/pythonlibs.git

#공통함수 임포트
from pythonlibs.torch_lib1 import *

#공통함수 체크
print(README)

ModuleNotFoundError: No module named 'pythonlibs'

## 학습 데이터 읽어오기

In [ ]:
import os
# 학습용 데이터 수집하기 (12장)

data_dir="dog_wolf"
train_dir = os.path.join(data_dir,'train')
test_dir = os.path.join(data_dir,'test')

classes = ['dog','wolf']

In [ ]:
#학습용 데이터 준비하기 (8장,10장,12장)

#transform 의 정의
#검증 데이터용 : 정규화만을 실시
test_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(0.5,0.5)

])

#학습 데이터용 : 정규화에 RandomHorizontalFlip 과 RandomErasing 을 추가로 실시
#RandomHorizontalFlip : 기본 이미지를 랜덤하게 회전해서 데이터 셋에 넣는것 => 더욱 다양한 데이터셋을 구성할 수 있음
#RandomErasing : 기본 이미지에서 특정 픽셀을 선택하고 지우기
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(0.5,0.5),
    transforms.RandomErasing(p=0.5,scale=(0.02,0.33),ratio=(0.3,3.3),value=0,inplace=False)

])

#dataset 정의
train_data = datasets.ImageFolder(train_dir,transform=train_transform)
train_data2 = datasets.ImageFolder(train_dir,transform=train_transform)
test_data = datasets.ImageFolder(test_dir,transform=test_transform)

#dataloader 정의
batch_size = 5
#train_loader = DataLoader(train_data, batch_size=batch_size,shuffle=True)
train_loader = DataLoader(train_data, batch_size=batch_size,shuffle=True)
train_loader2 = DataLoader(train_data2, batch_size=40,shuffle=False)
test_loader = DataLoader(train_data, batch_size=batch_size,shuffle=False)
test_loader2 = DataLoader(train_data, batch_size=10,shuffle=True)



## 학습 이미지 데이터 확인

In [ ]:
#show_images_labels : 각 이미지를 학습시킨 결과를 출력하는 함수, 뒤에 None 2개가 순서대로 model,device임
show_images_labels(train_loader2,classes,None,None)
#vsc 에선 plt img 가 안나옴

## 검증 이미지 데이터 확인

In [ ]:
#%%
# 검증용 데이터 (10건)
# 우선 정답 라벨 없이 이미지만을 표시합니다.
# 각 사진이 개와 늑대중 어떤 것인지 각자 판단해 보기 바랍니다.
torch_seed()
show_images_labels(test_loader2, ['', ''], None, None)
#vsc 에선 plt img 가 안나옴

In [ ]:
# 검증용 데이터 (10건)
# 이것이 정답 데이터 입니다.
# 10건 중 몇 건의 정답을 맞췄습니까.
torch_seed()
show_images_labels(test_loader2, classes, None, None)
#vsc 에선 plt img 가 안나옴

## 모델 구축

In [ ]:
# 사전에 학습이 끝난 모델을 이용해서 모델을 구축 (11장)

# 사전에 학습이 끝난 모델 읽어오기
from torchvision import models
net = models.vgg19_bn(pretrained = True)

# 전이 학습을 위해 required_grad 그래프를 False 로 설정
for param in net.parameters():
    param.requires_grad = False

# 난수 고정
torch_seed()

# 마지막 노드의 출력을 2로 변경
in_features = net.classifier[6].in_features
net.classifier[6] = nn.Linear(in_features, 2)

# AdaptiveAvgPool2d 함수 제거
net.avgpool = nn.Identity()

# GPU 사용
net = net.to(device)

# 학습률
lr = 0.001

# 손실 함수로 교차 엔트로피 사용
criterion = nn.CrossEntropyLoss()

# 최적화함수 정의

# 파라미터 수정 대상을 마지막 노드로 제한함
optimizer = optim.SGD(net.classifier[6].parameters(),lr=lr,momentum=0.9)

# history 파일을 동시에 초기화
history = np.zeros((0, 5))

## 학습

In [ ]:
# 학습(9장)
# 반복 횟수
num_epochs = 10

# 학습 (공통함수)
history = fit(net, optimizer, criterion, num_epochs, train_loader, test_loader, device, history)

In [ ]:
# 검증 데이터를 대상으로 결과 확인 (9장 공통함수)
torch_seed()
show_images_labels(test_loader2, classes, net, device)